In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
# If code is in the drive, the following line works, otherwise add folder names
os.chdir('/content/drive/My Drive/Code/Data')

Mounted at /content/drive


In [2]:
!pip3 install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import os
import numpy as np
import re

import tensorflow as tf
from sentence_transformers import SentenceTransformer
import pickle

In [4]:
# Google colab test code
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# Load in previous dictionairy of dataframes
with open('sparks_dfs.pkl', 'rb') as fp:
    pdfs = pickle.load(fp)

In [6]:
len(pdfs)

6574

In [7]:
cd translations

/content/drive/MyDrive/Code/Data/translations


In [8]:
# Gather all file names
names = os.listdir()

In [9]:
# Create list of all pickle saved dictionairies
opensubs = []
for name in names:
    if 'translation' in name and 'pkl' in name:
        with open(name,'rb') as fp:
            opensubs.append(pickle.load(fp))

In [10]:
# Create a dictionairy of all translated
OpenSubtitles = {}
for d in opensubs:
    for key in list(d.keys()):
        columns = d[key].columns
        if 'Sentence' in columns:
            OpenSubtitles[key] = d[key]

In [11]:
# Shows number of files of cleaned OpenSubtitles data, number of files after translations and the difference respectively
diff =set(pdfs.keys()) - set(OpenSubtitles.keys())
len(pdfs),len(OpenSubtitles),len(diff)

(6574, 5976, 600)

In [12]:
cd ..

/content/drive/MyDrive/Code/Data


In [ ]:
# Save frames with translations
#with open('WTranslationsOpenSubtitles.pkl', 'wb') as fp:
#    pickle.dump(OpenSubtitles,fp)

In [13]:
# Load in previous dictionairy of dataframes
with open('WTranslationsOpenSubtitles.pkl', 'rb') as fp:
    pdfs = pickle.load(fp)

In [14]:
frames = [pdfs[key] for key in pdfs.keys()]

In [21]:
# Needed for splitting the data
lens = [len(frame) for frame in frames]
senslens = [df['Sentence Length'].sum() for df in frames]
sum(senslens)

28067199

In [15]:
%%time
# Create one concattenated frame of alle separate frames (subtitle files of series and movies)
concatted = pd.DataFrame()
for frame in frames:
  concatted = pd.concat([concatted,frame],ignore_index=True)

concatted

CPU times: user 11min 21s, sys: 2min 32s, total: 13min 53s
Wall time: 13min 47s


,Sentence,Speaker,Next Speaker,Y,Sentence Length,NL translation
0,Hey - - sorry .,Rider,Kat,1,5,Hey sorry .
1,"I said , leave it !",Kat,Bianca,1,6,Ik zei: laat het!
2,Did you change your hair ?,Bianca,Chastity,1,6,Heb je je haar veranderd?
3,No .,Chastity,Girl,1,2,Nee .
4,Awesome shoes .,Girl,Miss Perky,1,3,Geweldige schoenen.
...,...,...,...,...,...,...
3437397,He gets dunked in the water again .,Peck,Peck,0,8,Hij wordt opnieuw in het water ondergedompeld.
3437398,Obviously not .,Peck,Peck,0,3,Duidelijk niet .
3437399,Dare I bring up.a sore subject .,Peck,Peck,0,7,Durf ik een pijnlijk onderwerp ter sprake te b...
3437400,Can we do a little business on the blackberry ...,Peck,Peck,0,11,Kunnen we wat zaken doen met de Blackberry-pri...


In [16]:
# Save frame
#with open('WTranslationsConcatOS.pkl', 'wb') as fp:
#    pickle.dump(concatted,fp)
#    print('Saved')

In [17]:
# Load in previous dictionairy of dataframes, that need to be converterd to spark dataframes
with open('WTranslationsConcatOS.pkl', 'rb') as fp:
    concatted = pickle.load(fp)

## Splitting the data

In [18]:
# Function to split data on percentages
def split_data(data,per):
  train = len(data)*per
  if train % 2 !=0:
    train = int(train-train % 2)
  else:
    train = int(train)

  return data[:train], data[train:]

In [22]:
# Split based on frames and then split the concattenaded frame based on these indices
# First training and test split
train,test = split_data(frames,0.9)
indext = sum(lens[:len(train)])

# Then split the training set into a training set and validation set
train_train,validation_train = split_data(train,0.9)
indexv = sum(lens[:len(train_train)])

#display(train_train[0])
#display(train_train[-1])
len(train),len(test)

(5378, 598)

In [ ]:
# This is to later compare the lenghts
train = concatted[:indext]
train

,Sentence,Speaker,Next Speaker,Y,Sentence Length,NL translation
0,Hey - - sorry .,Rider,Kat,1,5,Hey sorry .
1,"I said , leave it !",Kat,Bianca,1,6,Ik zei: laat het!
2,Did you change your hair ?,Bianca,Chastity,1,6,Heb je je haar veranderd?
3,No .,Chastity,Girl,1,2,Nee .
4,Awesome shoes .,Girl,Miss Perky,1,3,Geweldige schoenen.
...,...,...,...,...,...,...
2978839,"That 's a very good question , sir , and with ...",Sigmund,Sigmund,0,28,"Dat is een hele goede vraag, meneer, en met al..."
2978840,I 'm going to tell the sisters the truth .,Sigmund,Gideon,1,10,Ik ga de zusters de waarheid vertellen.
2978841,"I 'm sorry , old friend .",Gideon,Gideon,0,7,"Het spijt me , oude vriend ."
2978842,"But this too , is for the greater good .",Gideon,Gideon,0,10,Maar ook dit is voor het grotere goed.


In [ ]:
# Create test set from concatenated frame, based on previously found indices
test = concatted[indext:]
test

,Sentence,Speaker,Next Speaker,Y,Sentence Length,NL translation
2978844,"You know , your time travel section is due for...",Chris,Chris,0,14,"Weet je , je tijdreissectie is toe aan een ser..."
2978845,"I mean , I can find more information Googling .",Chris,Gideon,1,10,"Ik bedoel, ik kan meer informatie vinden door ..."
2978846,Googling ?,Gideon,Chris,1,2,Googlen?
2978847,Never mind . ( Chris steps down . ) Any luck w...,Chris,Paige,1,16,Laat maar zitten . (Chris treedt af.) Nog enig...
2978848,You mean since the last two minutes you asked ...,Paige,Chris,1,11,Je bedoelt sinds de laatste twee minuten dat j...
...,...,...,...,...,...,...
3437397,He gets dunked in the water again .,Peck,Peck,0,8,Hij wordt opnieuw in het water ondergedompeld.
3437398,Obviously not .,Peck,Peck,0,3,Duidelijk niet .
3437399,Dare I bring up.a sore subject .,Peck,Peck,0,7,Durf ik een pijnlijk onderwerp ter sprake te b...
3437400,Can we do a little business on the blackberry ...,Peck,Peck,0,11,Kunnen we wat zaken doen met de Blackberry-pri...


In [ ]:
# Create training set from concatenated frame, based on previously found indices
train_train = concatted[:indexv]
train_train

,Sentence,Speaker,Next Speaker,Y,Sentence Length,NL translation
0,Hey - - sorry .,Rider,Kat,1,5,Hey sorry .
1,"I said , leave it !",Kat,Bianca,1,6,Ik zei: laat het!
2,Did you change your hair ?,Bianca,Chastity,1,6,Heb je je haar veranderd?
3,No .,Chastity,Girl,1,2,Nee .
4,Awesome shoes .,Girl,Miss Perky,1,3,Geweldige schoenen.
...,...,...,...,...,...,...
2613240,You two stay back !,Swat On Radio,Swat On Radio,0,5,Jullie twee blijven achter!
2613241,Hold position !,Swat On Radio,Swat On Radio,0,3,Positie houden !
2613242,Man :,Swat On Radio,Swat On Radio,0,2,Man :
2613243,Hello ?,Swat On Radio,Swat On Radio,0,2,Hallo ?


In [ ]:
# Create validation set from concatenated frame, based on previously found indices
train_val = concatted[indexv:test.index[0]]
train_val

,Sentence,Speaker,Next Speaker,Y,Sentence Length,NL translation
2613245,He 's still sleeping it off .,Nick,Nick,0,7,Hij slaapt het nog steeds uit.
2613246,So you went to the cabin by yourself ?,Isabelle,Nick,1,9,Dus je ging alleen naar de hut?
2613247,You said you 'd come with me .,Ben,Isabelle,1,8,Je zei dat je met mij mee zou komen.
2613248,You hate it there .,Whispering,Ben,1,5,Je haat het daar.
2613249,"Copy , 19-2 .",Man On Police Radio,Whispering,1,4,"Kopieer , 19-2 ."
...,...,...,...,...,...,...
2978839,"That 's a very good question , sir , and with ...",Sigmund,Sigmund,0,28,"Dat is een hele goede vraag, meneer, en met al..."
2978840,I 'm going to tell the sisters the truth .,Sigmund,Gideon,1,10,Ik ga de zusters de waarheid vertellen.
2978841,"I 'm sorry , old friend .",Gideon,Gideon,0,7,"Het spijt me , oude vriend ."
2978842,"But this too , is for the greater good .",Gideon,Gideon,0,10,Maar ook dit is voor het grotere goed.


In [ ]:
# Short check if lenghts line up
print('Train, and train train and val com',len(train),len(train_train)+len(train_val))
print('Concatted, test and train and test com',len(concatted),len(test),len(train)+len(test))

Train, and train train and val com 2978844 2978844
Concatted, test and train and test com 3437402 458558 3437402


In [ ]:
# Save test splits
#with open('WTLOpenSubtitlesTrainingSet.pkl', 'wb') as fp:
 #   pickle.dump(train_train,fp)

#with open('WTLOpenSubtitlesValidationSet.pkl', 'wb') as fp:
 #   pickle.dump(train_val,fp)

#with open('WTLOpenSubtitlesTestSet.pkl', 'wb') as fp:
 #   pickle.dump(test,fp)

print('Saved')

Saved


In [ ]:
# Save all labels
#with open('YSWTLOpenSubtitlesTrainingSet.pkl', 'wb') as fp:
#    pickle.dump([np.array(train_train.Y.tolist(),dtype=np.float32),np.array(train_val.Y.tolist(),dtype=np.float32),np.array(test.Y.tolist(),dtype=np.float32)],fp)
#    print('Saved')

Saved


In [23]:
with open('WTLOpenSubtitlesTrainingSet.pkl', 'rb') as fp:
    train_train = pickle.load(fp)

with open('WTLOpenSubtitlesValidationSet.pkl', 'rb') as fp:
    train_val = pickle.load(fp)

with open('WTLOpenSubtitlesTestSet.pkl', 'rb') as fp:
    test = pickle.load(fp)

In [ ]:
# Label (im)balance per set
print('Training set 1 ',len(train_train[train_train.Y == 1]),'0',len(train_train[train_train.Y == 0]),'total',len(train_train))
print('Validation set 1 ',len(train_val[train_val.Y == 1]),'0',len(train_val[train_val.Y == 0]),'total',len(train_val))
print('Test set 1 ',len(test[test.Y == 1]),'0',len(test[test.Y == 0]),'total',len(test))

Training set 1  1329883 0 1283362 total 2613245
Validation set 1  185377 0 180222 total 365599
Test set 1  241524 0 217034 total 458558


## Create the BERT sentence embeddings

In [25]:
def create_embeddings(text,model):
  model = SentenceTransformer(model)
  return model.encode(text)

In [26]:
BERT_model = 'paraphrase-multilingual-mpnet-base-v2'

In [24]:
X_train = train_train
X_valid = train_val

In [ ]:
%%time
X_train = np.array(create_embeddings(train_train.Sentence.tolist(),BERT_model),dtype=np.float32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

CPU times: user 48min 27s, sys: 1min 25s, total: 49min 53s
Wall time: 45min 7s


In [ ]:
# Save train embedding
#with open('WTLOpenSubtitlesTrainEmbeddings.pkl', 'wb') as fp:
#    pickle.dump(X_train,fp)
#    print('Saved')

Saved


In [27]:
%%time
X_valid = np.array(create_embeddings(train_val.Sentence.tolist(),BERT_model),dtype=np.float32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 6min 46s, sys: 12.6 s, total: 6min 59s
Wall time: 6min 32s


In [ ]:
# Load in previous dictionairy of dataframes, that need to be converterd to spark dataframes
#with open('WTLOpenSubtitlesValidEmbeddings.pkl', 'wb') as fp:
#    pickle.dump(X_valid,fp)
#    print('Saved')

Saved


## Dutch BERTs

In [ ]:
text = train_train['NL translation'].astype(str).tolist()

In [ ]:
%%time
X_train = np.array(create_embeddings(text,BERT_model),dtype=np.float32)

CPU times: user 44min 59s, sys: 1min 16s, total: 46min 15s
Wall time: 41min 42s


In [ ]:
# Save Dutch training embeddings
#with open('DutchWTLOpenSubtitlesTrainEmbeddings.pkl', 'wb') as fp:
#    pickle.dump(X_train,fp)
#    print('Saved')

Saved


In [ ]:
Dutch_text = train_val['NL translation'].astype(str).tolist()

In [ ]:
%%time
X_valid = np.array(create_embeddings(Dutch_text,BERT_model),dtype=np.float32)

CPU times: user 6min 9s, sys: 10.2 s, total: 6min 20s
Wall time: 5min 40s


In [ ]:
# Load in previous dictionairy of dataframes, that need to be converterd to spark dataframes
with open('DutchWTLOpenSubtitlesValidEmbeddings.pkl', 'wb') as fp:
    pickle.dump(X_valid,fp)
    print('Saved')

Saved
